# 教程 1: 数据获取与探索 / Tutorial 1: Data Fetching and Exploration

本教程将教你如何从币安获取历史数据并进行基本的探索性数据分析。

This tutorial will teach you how to fetch historical data from Binance and perform basic exploratory data analysis.

## 1. 导入库 / Import Libraries

In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 设置 matplotlib 支持中文显示 / Set matplotlib to support Chinese display
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 添加项目路径 / Add project path
sys.path.append('..')

from utils.binance_client import BinanceUtility

print("库导入成功！/ Libraries imported successfully!")

## 2. 从币安获取数据 / Fetch Data from Binance

我们将获取比特币 (BTC) 过去 6 个月的小时级数据。
We will fetch the past 6 months of hourly data for Bitcoin (BTC).

In [ ]:
# 创建币安客户端 / Create Binance client
client = BinanceUtility()

# 获取 BTC/USDT 数据 / Fetch BTC/USDT data
symbol = 'BTCUSDT'
interval = '1h'  # 小时级数据 / hourly data
start_time = '6 months ago UTC'

print(f"正在获取 {symbol} 的数据... / Fetching {symbol} data...")
df = client.fetch_historical_data(symbol, interval, start_time)

print(f"\n数据形状 / Data shape: {df.shape}")
print(f"\n前 5 行数据 / First 5 rows:")
df.head()

## 3. 数据基本信息 / Basic Data Information

In [ ]:
# 数据类型 / Data types
print("数据类型 / Data Types:")
print(df.dtypes)

# 缺失值检查 / Missing values check
print("\n缺失值统计 / Missing Values:")
print(df.isnull().sum())

# 统计摘要 / Statistical summary
print("\n统计摘要 / Statistical Summary:")
df.describe()

## 4. 可视化价格走势 / Visualize Price Trends

In [ ]:
# 设置时间索引 / Set timestamp as index
df_indexed = df.set_index('timestamp')

# 绘制收盘价 / Plot closing price
plt.figure(figsize=(14, 6))
plt.plot(df_indexed.index, df_indexed['close'], label='收盘价 / Close Price', linewidth=1)
plt.title(f'{symbol} 价格走势 / {symbol} Price Trend (6 Months)', fontsize=14)
plt.xlabel('时间 / Time', fontsize=12)
plt.ylabel('价格 / Price (USDT)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. 分析成交量 / Analyze Volume

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# 价格图 / Price chart
ax1.plot(df_indexed.index, df_indexed['close'], label='收盘价 / Close', color='blue', linewidth=1)
ax1.set_ylabel('价格 / Price', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_title(f'{symbol} 价格与成交量 / {symbol} Price and Volume', fontsize=14)

# 成交量图 / Volume chart
ax2.bar(df_indexed.index, df_indexed['volume'], label='成交量 / Volume', color='orange', alpha=0.6)
ax2.set_xlabel('时间 / Time', fontsize=12)
ax2.set_ylabel('成交量 / Volume', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. 计算收益率 / Calculate Returns

收益率是价格变动的百分比，是分析市场波动的重要指标。
Returns are the percentage change in price, an important indicator for analyzing market volatility.

In [ ]:
# 计算每小时收益率 / Calculate hourly returns
df_indexed['returns'] = df_indexed['close'].pct_change()

# 绘制收益率 / Plot returns
plt.figure(figsize=(14, 6))
plt.plot(df_indexed.index, df_indexed['returns'], label='收益率 / Returns', linewidth=1, alpha=0.7)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.title(f'{symbol} 每小时收益率 / {symbol} Hourly Returns', fontsize=14)
plt.xlabel('时间 / Time', fontsize=12)
plt.ylabel('收益率 / Returns', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. 收益率分布分析 / Returns Distribution Analysis

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 直方图 / Histogram
ax1.hist(df_indexed['returns'].dropna(), bins=100, color='skyblue', edgecolor='black', alpha=0.7)
ax1.set_xlabel('收益率 / Returns', fontsize=12)
ax1.set_ylabel('频数 / Frequency', fontsize=12)
ax1.set_title('收益率分布直方图 / Returns Histogram', fontsize=14)
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax1.grid(True, alpha=0.3)

# 箱线图 / Box plot
ax2.boxplot(df_indexed['returns'].dropna(), vert=True)
ax2.set_ylabel('收益率 / Returns', fontsize=12)
ax2.set_title('收益率箱线图 / Returns Box Plot', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"收益率统计 / Returns Statistics:")
print(f"均值 / Mean: {df_indexed['returns'].mean():.6f}")
print(f"标准差 / Std: {df_indexed['returns'].std():.6f}")
print(f"最大值 / Max: {df_indexed['returns'].max():.6f}")
print(f"最小值 / Min: {df_indexed['returns'].min():.6f}")

## 8. 保存数据 / Save Data

将处理后的数据保存到本地，供后续分析使用。
Save the processed data locally for future analysis.

In [ ]:
# 创建 data 目录 / Create data directory
if not os.path.exists('../data'):
    os.makedirs('../data')

# 保存数据 / Save data
file_path = f'../data/{symbol}_hist.csv'
df.to_csv(file_path, index=False)
print(f"数据已保存至 / Data saved to: {file_path}")

## 总结 / Summary

在本教程中，我们学习了：
1. 如何从币安 API 获取历史 K 线数据
2. 如何检查数据的基本信息和质量
3. 如何可视化价格走势和成交量
4. 如何计算和分析收益率
5. 如何保存数据供后续使用

In this tutorial, we learned:
1. How to fetch historical K-line data from Binance API
2. How to check basic information and quality of data
3. How to visualize price trends and volume
4. How to calculate and analyze returns
5. How to save data for future use

下一步，我们将学习特征工程！/ Next, we'll learn feature engineering!